In [ ]:


import pandas as pd
from sklearn.model_selection import train_test_split

file_path = 'Laptop_price.csv' 
df = pd.read_csv(file_path)

X = df.drop(columns=['Price'])
y = df['Price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

df.head()

,Brand,Processor_Speed,RAM_Size,Storage_Capacity,Screen_Size,Weight,Price
0,Asus,3.830296,16,512,11.185147,2.641094,17395.093065
1,Acer,2.912833,4,1000,11.311372,3.260012,31607.605919
2,Lenovo,3.241627,4,256,11.853023,2.029061,9291.023542
3,Acer,3.806248,16,512,12.280360,4.573865,17436.728334
4,Acer,3.268097,32,1000,14.990877,4.193472,32917.990718


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
import joblib
from sklearn.metrics import mean_squared_error


num_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_features = X.select_dtypes(include=['object']).columns.tolist()

num_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

cat_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer([
    ('num', num_transformer, num_features),
    ('cat', cat_transformer, cat_features)
])
ю
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor(n_estimators=100, random_state=42)) 
])

pipeline.fit(X_train, y_train)

joblib.dump(pipeline, 'laptop_price_model.pkl')

y_pred = pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

loaded_model = joblib.load('laptop_price_model.pkl')
y_pred_loaded = loaded_model.predict(X_test)
print("Предсказания загруженной модели:", y_pred_loaded)

Mean Squared Error: 40582.32261422354
Предсказания загруженной модели: [10571.78908008 31487.51968542  9674.64055682  9211.8636902
 32003.34846338 31557.19556874 10636.96086722 16830.91457736
 18555.01523317 32351.40832108 10048.91839452 10684.48470125
 32933.57072207  9233.05368659 17134.1983124   9077.69815693
 32075.74637496 32276.03160453 10582.50901113 31802.77791057
 10551.09712512 10767.67109008  9178.44365052 10589.53670212
  9530.24931832 17666.38321506 10664.21058468 18221.81828545
 10839.48031885 32619.27296611 16720.25747304 32776.90752368
 16654.8739964  32027.87679853 31695.56980996 10323.74248587
 31266.86266937  9072.91817215 32609.05728979 17352.70843304
 32040.36700799  9605.43399131 17694.47199275 31680.89835971
 17238.06831179 16814.6857629  32737.99372119 16875.47344575
  9738.61780967 17085.40169797 16766.25281097 31385.02491432
  9739.95096948 10464.20587747 32075.38452167 32917.66002967
 31435.62543319 31635.4169755   9122.39844736 32347.53586834
 17549.53545788

In [1]:

!git init
!git add .
!git commit -m "Добавлен ML-пайплайн"
!git remote add origin https://github.com/Rinotter/rina33.git
!git push -u origin main

Initialized empty Git repository in /Users/polina/Documents/лаб питон/.git/
[master (root-commit) d3367f8] Добавлен ML-пайплайн
 3 files changed, 2406 insertions(+)
 create mode 100644 Laptop_price.csv
 create mode 100644 Laptop_price_secured.csv
 create mode 100644 main1.ipynb
error: src refspec main does not match any.
error: failed to push some refs to 'https://github.com/Rinotter/rina33.git'


In [ ]:
%%writefile app.py
from fastapi import FastAPI, File, UploadFile
import pandas as pd
import joblib
from io import BytesIO

app = FastAPI()

model_path = "laptop_price_model.pkl"
model = joblib.load(model_path)

@app.post("/predict/")
async def predict(file: UploadFile = File(...)):
    content = await file.read()
    df = pd.read_csv(BytesIO(content))
    predictions = model.predict(df)
    return {"predictions": predictions.tolist()}


Overwriting app.py


In [ ]:

!pip install python-multipart

!uvicorn app:app --reload

from pyngrok import ngrok


ngrok.set_auth_token("2ws575rPxJ22OS0A3NaOYyyqhBx_84oPHoikzYEpZGTX1qyqH")

# Создание туннеля для порта 8000
public_url = ngrok.connect(8000)
print("API доступно по адресу:", public_url)

INFO:     Will watch for changes in these directories: ['/Users/polina/Downloads/лаб питон']
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [69388] using StatReload
INFO:     Started server process [69390]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     127.0.0.1:57988 - "GET / HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:57987 - "GET / HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:58356 - "POST /predict/ HTTP/1.1" 200 OK
^C
INFO:     Shutting down
INFO:     Finished server process [69390]
INFO:     Stopping reloader process [69388]
API доступно по адресу: NgrokTunnel: "https://e78e-143-244-44-179.ngrok-free.app" -> "http://localhost:8000"


curl -X 'POST' \
  'http://127.0.0.1:8000/predict/' \
  -H 'accept: application/json' \
  -H 'Content-Type: multipart/form-data' \
  -F 'file=@Laptop_price.csv;type=text/csv'

  